In [59]:
import pandas as pd
import openpyxl
import numpy as np
from torch.utils import data
import torch
from d2l import torch as d2l
from torch import nn
from numpy.random import permutation
import pandas as pd
import random
import numpy as np
from sklearn.utils import shuffle

# hyperparameter setting:
TrainRatio = 4/5
ValiRatio = 1/5

# read csv
# fpath = "./datas/ml-latest-small/ratings.csv" # 注意当前路径需要是datas上一级路径
# DataSet = pd.read_csv(fpath)
# read xlsx
fpath = "/Users/yun/Desktop/OA_残值估值/中汽研二手车数据/feature_1020.xlsx"
DataSet = pd.read_excel(fpath)

# 数据清洗，离群点去噪
def Outlier_Delete(DataSet, Series_name ,t_stud=3):
    '''
    按照正态分布的末端值去除离群点
    Input：DataFrame DataSet ； 针对对象列； t-student 的值，一般取3，可调整正常值范围
    Output：用该对象列去除离群点后的数据集
    '''
    DataSet = DataSet.sort_values(by=Series_name)
    mean = DataSet[Series_name].mean()
    sigma = DataSet[Series_name].std()
    # 筛选异常值
    DataSet_new = DataSet.loc[( mean-t_stud*sigma < DataSet[Series_name]) & (DataSet[Series_name]  < mean+t_stud*sigma) , :]
    return DataSet_new

DataSet_new = Outlier_Delete(DataSet, '新车指导价', 0.618)
# 以新车指导价对样本进行去噪，留下大概只有指导价150w以内的样本
DataSet_new2 = Outlier_Delete(DataSet_new, '车龄', 0.618)
# 以车龄对样本进行去噪，留下大概只有19年以内的样本
DataSet_new3 = Outlier_Delete(DataSet_new2, '过户次数', 6)
# 以过户次数对样本进行去噪，留下大概只有指导价5次以内的样本
DataSet = Outlier_Delete(DataSet_new3, '平均里程', 4)
# 以平均里程数对样本进行去噪，留下大概只有指导价5次以内的样本
# 以上留下多少，用Outlier_Delete最后一个参数可调整
DataSet = shuffle(DataSet)



nData = len(DataSet)
nTrain = TrainRatio * nData
nVali = ValiRatio * nData
train_data = DataSet.iloc[0:int(nTrain)]
test_data =DataSet.iloc[int(nTrain):nData]
# test_data =DataSet[nTrain+nVali:-1]
guideprice_test = test_data['新车指导价'].values.reshape(-1, 1)

# testset的新车指导价
# preprocess
all_features = pd.concat((train_data.iloc[:, 0:-1], test_data.iloc[:, 0:-1]))
print(all_features)


        新车指导价       车系 变速器类型  驱动形式 国产进口合资 燃料类型 交易城市 价格类型  车龄      平均里程  过户次数
239609   7.99       逸动    手动  前置前驱     国产   汽油   青岛  B2C   6  1.000000     0
353260  36.99  凯迪拉克XTS    自动  前置前驱     合资   汽油   昆明  C2B   4  1.095000     0
265397  43.99  牧马人(进口)    自动  前置四驱     进口   汽油   临沂  B2C   6  1.333333     0
122804  15.70    新世代全顺    手动  前置后驱     合资   柴油   武汉  B2C   4  0.750000     0
61114   46.80     奔驰E级    自动  前置后驱     合资   汽油   广州  B2C   5  1.638000     0
...       ...      ...   ...   ...    ...  ...  ...  ...  ..       ...   ...
226552  13.08       雷凌  无级变速  前置前驱     合资   汽油  哈尔滨  B2C   4  0.732500     0
33209   13.48      卡罗拉  无级变速  前置前驱     合资   汽油   钦州  B2C   5  1.240000     0
62604   48.80     奔驰E级    自动  前置后驱     合资   汽油   深圳  C2B   5  0.760000     0
42852   41.53    奥迪A6L   双离合  前置前驱     合资   汽油   潍坊  B2C   4  2.400000     0
177895   7.48       雨燕    自动  前置前驱     合资   汽油   廊坊  B2B   5  1.544000     1

[159389 rows x 11 columns]


In [60]:
# process numerical variable
# normalization
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
# feature为数值的列索引, 注意object即为字符串类型
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后均值为0，因此我们可以将缺失值设置为0（均值）
all_features[numeric_features] = all_features[numeric_features].fillna(0)
# process discrete variable
all_features = pd.get_dummies(all_features, dummy_na=True)
# One-hot code
# `Dummy_na=True` 将“na”（缺失值）视为有效的特征值，并为其创建指示符特征。


train_features = torch.tensor(all_features.iloc[0:int(nTrain)].values, dtype=d2l.float32)
test_features = torch.tensor(all_features.iloc[int(nTrain):].values, dtype=d2l.float32)
# 通过value属性提取pandas中的numpy格式并将其转换为torch中的张量, 注意我们已经把所有类型都转成数值型数据了。
train_labels = torch.tensor(
    train_data['交易价格'].values.reshape(-1, 1), dtype=d2l.float32)
test_labels = torch.tensor(
    test_data['交易价格'].values.reshape(-1, 1), dtype=d2l.float32)
# 提取标签并转换成列张量

# 定义模型
loss = nn.MSELoss()

# 定义损失函数
in_features = train_features.shape[1]
# feature数量

def get_net():
    net = nn.Sequential(nn.Linear(in_features,1))
    return net

# 定义线性模型：单层网络，全连接层单输出；
def log_rmse(net, features, labels):
    # 为了在取对数时进一步稳定该值，将小于1的值设置为1
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds),
                           torch.log(labels)))
    return rmse.item()
# 定义衡量指标，注意我们还是用MSE当作损失函数来优化，这个log_rmse只是衡量指标。

# 定义训练器
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    '''

    :param net: 定义好的神经网络
    :param train_features: 训练集（不是原始数据集）的feature
    :param train_labels: 训练集（不是原始数据集）的label
    :param test_features: 测试集（但实际上经常输入验证集）的feature
    :param test_labels:测试集（但实际上经常输入验证集）的label
    :param num_epochs:迭代次数
    :param learning_rate:学习率
    :param weight_decay:
    :param batch_size:批量大小
    :return:每大步的训练lgrmse 和 测试lgrmse
    '''
    train_ls, test_ls = [], [] # 用来记录每步的logRMSE
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    # 这里使用的是Adam优化算法
    optimizer = torch.optim.Adam(net.parameters(),
                                 lr = learning_rate,
                                 weight_decay = weight_decay)
    for epoch in range(num_epochs):
        for X, y in train_iter:
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None: # 如果有testset输入近来的话
            test_ls.append(log_rmse(net, test_features, test_labels))
    test_pred = net(test_features)
    test_pred = test_pred.detach().numpy()

    return train_ls, test_ls, test_pred

# 为K-fold 划分数据集
def get_k_fold_data(k, i, X, y):
    '''
    为K-fold 划分数据集
    :param k: k-fold 的k
    :param i: 第i个fold作为validation set
    :param X: 原始训练集（未划分训练验证的）的整个feature
    :param y: 原始训练集的整个label
    :return: 交叉检验使用的训练集和验证集
    '''
    assert k > 1
    fold_size = X.shape[0] // k # 每个fold的规模
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size) # 一个fold的索引
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
        # i-th fold是validation set，其他作为TrainSet
    return X_train, y_train, X_valid, y_valid

def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay,
           batch_size):
    '''

    :param k: number of folds
    :param X_train: total train set features
    :param y_train: total train labels
    :param num_epochs: No. of displays
    :param learning_rate: learning rate
    :param weight_decay:
    :param batch_size: size of batch
    :return: Trainlogmse和Validationlogmse的平均值（把k次算到的取平均，更重要的是后者）
    '''
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        # 依次对每一个fold当作测试集，剩余的当作训练集
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls,_ = train(net, *data, num_epochs, learning_rate,
                                   weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        if i == 0:
            d2l.plot(list(range(1, num_epochs + 1)), [train_ls, valid_ls],
                     xlabel='epoch', ylabel='lgrmse', xlim=[1, num_epochs],
                     legend=['train', 'valid'], yscale='log')
        print(f'fold {i + 1}, train log rmse {float(train_ls[-1]):f}, '
              f'valid log rmse {float(valid_ls[-1]):f}')
    return train_l_sum / k, valid_l_sum / k

def train_and_pred(train_features, test_features, train_labels, test_labels,
                   num_epochs, lr, weight_decay, batch_size):
    net = get_net()
    train_ls, test_ls,test_pred = train(net, train_features, train_labels, test_features, test_labels,
                        num_epochs, lr, weight_decay, batch_size)
    return train_ls, test_ls, test_pred

__name__ = 'TrainPred'
if __name__ == 'CrossValidation':
    k, num_epochs, lr, weight_decay, batch_size = 5, 100, 0.05, 0, 1000
    train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                              weight_decay, batch_size)
    print(f'{k}-折验证: 平均训练log rmse: {float(train_l):f}, '
          f'平均验证log rmse: {float(valid_l):f}')

elif __name__ == 'TrainPred':
    k, num_epochs, lr, weight_decay, batch_size = 5, 100, 0.05, 0, 1000
    train_ls, test_ls, test_pred = train_and_pred(train_features, test_features, train_labels, test_labels,
                   num_epochs, lr, weight_decay, batch_size)
    print(f' 训练log rmse: {float(train_ls[-1]):f}, '
          f'测试log rmse: {float(test_ls[-1]):f},')
    test_pred = test_pred.ravel()
    print(test_pred)
    print(guideprice_test)
    # test_pred:numpy.ndarray
    # guideprice_test :numpy.ndarray
    
else:
    print('error main command')


 训练log rmse: 0.166212, 测试log rmse: 0.183189,
[ 8.340372   9.611574   3.7082052 ... 27.446644  25.006922   3.8791473]
[[16.78]
 [19.28]
 [12.48]
 ...
 [48.8 ]
 [41.53]
 [ 7.48]]


In [61]:
import math
Trade_price = pd.DataFrame(test_labels.numpy()) # trade price
print(Trade_price)
Guide_price = pd.DataFrame(guideprice_test) #guide price
Pred_price = pd.DataFrame(test_pred) # prediction price
print(Guide_price)
print(Pred_price)
data = pd.concat((Pred_price,Trade_price,Guide_price), axis = 1)
# gap = abs(pred_price-trade price)/guide price
print(data)

# write
writer = pd.ExcelWriter('A.xlsx')
data.to_excel(writer,'page_1',float_format = '%.5f')
writer.save()

gap = abs(Pred_price-Trade_price)/Guide_price
print(gap)
print(type(gap))
rate = sum(gap.values<=0.10)/(nData-nTrain)
print(rate)

               0
0       7.910000
1       9.500000
2       5.700100
3       4.250000
4       8.150000
...          ...
31873  10.700000
31874   7.680000
31875  27.000000
31876  26.639999
31877   5.310000

[31878 rows x 1 columns]
           0
0      16.78
1      19.28
2      12.48
3      10.99
4      12.98
...      ...
31873  13.08
31874  13.48
31875  48.80
31876  41.53
31877   7.48

[31878 rows x 1 columns]
               0
0       8.340372
1       9.611574
2       3.708205
3       2.739346
4       9.384337
...          ...
31873  10.257640
31874   8.907538
31875  27.446644
31876  25.006922
31877   3.879147

[31878 rows x 1 columns]
               0          0      0
0       8.340372   7.910000  16.78
1       9.611574   9.500000  19.28
2       3.708205   5.700100  12.48
3       2.739346   4.250000  10.99
4       9.384337   8.150000  12.98
...          ...        ...    ...
31873  10.257640  10.700000  13.08
31874   8.907538   7.680000  13.48
31875  27.446644  27.000000  48.80
31876  2

In [62]:
net = get_net()
w = net[0].weight.data
print('参数 w：', w.reshape(in_features)[0:5])
b = net[0].bias.data
print('bias term b：', b)

参数 w： tensor([-0.0114, -0.0070,  0.0276,  0.0032,  0.0226])
bias term b： tensor([-0.0157])
